In [1]:
import tensorflow as tf
from tensorflow.keras import layers,models,backend

In [2]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

# DDQN BC code

In [ ]:
if self.params['ddqn']:
            #DDQN
            #eval net
            A_prev = self.s
            for i in np.arange(self.params['evalnet_layer_A']):
                A_prev=layers.Dense(self.params['evalnet_A'][i]['num'], activation='relu', name='evalnet_A'+str(i))(A_prev)
            V_prev = self.s
            for i in np.arange(self.params['evalnet_layer_V']):
                V_prev=layers.Dense(self.params['evalnet_V'][i]['num'], activation='relu', name='evalnet_V'+str(i))(V_prev)
            A_prev_avg = A_prev-tf.reduce_mean(A_prev)
            self.eval_out = layers.Add()([V_prev,A_prev_avg])
            
            #target net
            An_prev = self.s_next
            for i in np.arange(self.params['targetnet_layer_A']):
                An_prev=layers.Dense(self.params['targetnet_A'][i]['num'], activation='relu', name='targetnet_A'+str(i))(An_prev)
            Vn_prev = self.s
            for i in np.arange(self.params['targetnet_layer_V']):
                Vn_prev=layers.Dense(self.params['targetnet_V'][i]['num'], activation='relu', name='targetnet_V'+str(i))(Vn_prev)
            An_prev_avg = An_prev-tf.reduce_mean(An_prev)
            self.target_out = layers.Add()([Vn_prev,An_prev_avg])
        else:
            

# DQN code

In [6]:
class DQN:
    
    def __init__(self,params,env):
        tf.compat.v1.disable_eager_execution()
        self.params=params
        self.memory_buffer = deque(maxlen=2000)
        self.action_table=pd.read_excel('./action_table_of_DQN.xlsx').values[:,1:]
        print('table shape: ',self.action_table.shape)
        
        self.model=_build_net()
        self.target_model=_build_net()
        self.update_target_model()
        
    def _build_net(self):
        self.s = layers.Input(shape=self.params['state_dim'],name='s_input')
        self.s_next = layers.Input(shape=self.params['state_dim'],name='s_next_input')
        self.q_target = layers.Input(shape=self.params['action_dim'],name='qt_input')
        
        #DQN
        #eval net
        V_prev = self.s
        for i in np.arange(self.params['evalnet_layer_V']):
            V_prev=layers.Dense(self.params['evalnet_V'][i]['num'], activation='relu', name='evalnet_V'+str(i))(V_prev)
        
        model=models.Model(inputs=[self.s,self.s_next,self.q_target],outputs=self.eval_out)
        return model
    
    def choose_action(self,state,train_log):
        #input state, output action
        if train_log:
            #epsilon greedy
            pa = np.random.uniform()
            if pa < self.params['epsilon']:
                action_value = self.model.predict([state])
                action = np.argmax(action_value)
            else:
                action = np.random.randit(0,self.params['action_dim'])
        else:
            action_value = self.model.predict([state])
            action = np.argmax(action_value)
        return action

    
    def remember(self, state, action, reward, next_state, done):
        item = (state, action, reward, next_state, done)
        self.memory_buffer.append(item)

    def process_batch(self, batch):
         # 从经验池中随机采样一个batch
        data = random.sample(self.memory_buffer, batch)
        # 生成Q_target。
        states = np.array([d[0] for d in data])
        next_states = np.array([d[3] for d in data])

        y = self.model.predict(states)
        q = self.target_model.predict(next_states)

        for i, (_, action, reward, _, done) in enumerate(data):
            target = reward
            if not done:
                target += self.gamma * np.amax(q[i])
            y[i][action] = target
        return states, y
    
    
    def train(self,total_step):
        #sampling and upgrading
        for j in range(self.params['training_step']):
            total_step=0
            for i in range(self.params['num_rain']):
                print('training step:',j,' sampling num:',i)
                #Sampling: each rainfall represent one round of sampling
                s = self.env.reset(self.RainData[i])
                done, batch = False, 0
                while not done:
                    a = self.choose_action(s,True)
                    action = self.action_table[a,:].tolist()
                    snext,reward,done = self.env.step(action)
                    self.remember(s, a, reward, snext, done)
                    s = snext
                    batch+=1
                
                #Upgrading: each rainfall for one round of upgrading
                X, y = self.process_batch(batch)
                loss = self.model.train_on_batch(X, y)
                count += 1
                # 减小egreedy的epsilon参数。
                if self.epsilon >= self.epsilon_min:
                    self.epsilon *= self.epsilon_decay
                # 固定次数更新target_model
                if count != 0 and count % 20 == 0:
                    self.target_model.set_weights(self.model.get_weights())
                if i % 5 == 0:
                    history['episode'].append(i)
                    history['Episode_reward'].append(reward_sum)
                    history['Loss'].append(loss)

                    print('Episode: {} | Episode reward: {} | loss: {:.3f} | e:{:.2f}'.format(i, reward_sum, loss, self.epsilon))

            self.model.save_weights('./model/dqn.h5')
        return history
    
    def play(self):
        observation = self.env.reset()

        count = 0
        reward_sum = 0
        random_episodes = 0

        while random_episodes < 10:
            self.env.render()

            x = observation.reshape(-1, 4)
            q_values = self.model.predict(x)[0]
            action = np.argmax(q_values)
            observation, reward, done, _ = self.env.step(action)

            count += 1
            reward_sum += reward

            if done:
                print("Reward for this episode was: {}, turns was: {}".format(reward_sum, count))
                random_episodes += 1
                reward_sum = 0
                count = 0
                observation = self.env.reset()

        self.env.close()

In [4]:
import SWMM_ENV

In [5]:
params={
    'state_dim':5,
    'action_dim':8,
    'evalnet_layer_A':3,
    'evalnet_A':[{'num':30},{'num':30},{'num':30}],
    'evalnet_layer_V':3,
    'evalnet_V':[{'num':30},{'num':30},{'num':30}],
    'targetnet_layer_A':3,
    'targetnet_A':[{'num':30},{'num':30},{'num':30}],
    'targetnet_layer_V':3,
    'targetnet_V':[{'num':30},{'num':30},{'num':30}],
    
}
env

In [48]:
model=DQN(params)

In [49]:
model._build_net()